# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,24.00,87,100,0.67,US,1688946834
1,1,olonkinbyen,70.9221,-8.7187,2.56,96,100,10.38,SJ,1688946834
2,2,waitangi,-43.9535,-176.5597,11.01,88,99,5.36,NZ,1688946835
3,3,pokrovsk,48.2820,37.1758,15.39,87,100,6.11,UA,1688946835
4,4,ketchikan,55.3422,-131.6461,18.26,68,20,6.17,US,1688946296


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points("Lng", 
                                       "Lat",
                                       size = "Humidity",
                                       geo =True, 
                                       tiles = "OSM", 
                                       frame_width = 800, 
                                       frame_height = 600,
                                       color = "City")
# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_reduced = city_data_df.loc[(city_data_df["Max Temp"] < 34) &\
                                     (city_data_df["Max Temp"] > 29) &\
                                     (city_data_df["Wind Speed"] < 4.5) &\
                                     (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
city_data_reduced = city_data_reduced.dropna(axis=0)

# Display sample data
city_data_reduced

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,bangar,16.8936,120.4243,29.25,66,0,0.60,PH,1688946851
96,96,kalaleh,37.3807,55.4916,31.39,15,0,3.50,IR,1688946865
180,180,newman,37.3138,-121.0208,30.03,10,0,3.38,US,1688946892
217,217,tambacounda,13.7539,-13.7586,33.03,66,0,2.06,SN,1688946904
278,278,metlili chaamba,32.2667,3.6333,33.52,13,0,0.00,DZ,1688946925
294,294,show low,34.2542,-110.0298,31.16,16,0,3.60,US,1688946930
327,327,kelso,46.1468,-122.9084,29.71,51,0,3.60,US,1688946941
363,363,patterson,37.4716,-121.1297,30.01,16,0,3.96,US,1688946951
390,390,balad,34.0164,44.1453,33.04,22,0,3.93,IQ,1688946960
480,480,charleston,32.7766,-79.9309,33.01,60,0,2.57,US,1688946989


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_reduced[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,bangar,PH,16.8936,120.4243,66,
96,kalaleh,IR,37.3807,55.4916,15,
180,newman,US,37.3138,-121.0208,10,
217,tambacounda,SN,13.7539,-13.7586,66,
278,metlili chaamba,DZ,32.2667,3.6333,13,
294,show low,US,34.2542,-110.0298,16,
327,kelso,US,46.1468,-122.9084,51,
363,patterson,US,37.4716,-121.1297,16,
390,balad,IQ,34.0164,44.1453,22,
480,charleston,US,32.7766,-79.9309,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "lang": "en",
    "categories":"accommodation.hotel"

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bangar - nearest hotel: Villa adelfa resort hotel and restaurant
kalaleh - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
tambacounda - nearest hotel: Le relais de Tamba
metlili chaamba - nearest hotel: No hotel found
show low - nearest hotel: Holiday Inn Express Showlow
kelso - nearest hotel: Best Western Aladdin Inn
patterson - nearest hotel: Hampton Inn & Suites
balad - nearest hotel: No hotel found
charleston - nearest hotel: Mills House Charleston, Curio Collection by Hilton
hurghada - nearest hotel: Sea Horse Hotel
naama - nearest hotel: Résidence des hôtes إقامة الضيوف


,City,Country,Lat,Lng,Humidity,Hotel Name
52,bangar,PH,16.8936,120.4243,66,Villa adelfa resort hotel and restaurant
96,kalaleh,IR,37.3807,55.4916,15,No hotel found
180,newman,US,37.3138,-121.0208,10,No hotel found
217,tambacounda,SN,13.7539,-13.7586,66,Le relais de Tamba
278,metlili chaamba,DZ,32.2667,3.6333,13,No hotel found
294,show low,US,34.2542,-110.0298,16,Holiday Inn Express Showlow
327,kelso,US,46.1468,-122.9084,51,Best Western Aladdin Inn
363,patterson,US,37.4716,-121.1297,16,Hampton Inn & Suites
390,balad,IQ,34.0164,44.1453,22,No hotel found
480,charleston,US,32.7766,-79.9309,60,"Mills House Charleston, Curio Collection by Hi..."


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)